<a href="https://colab.research.google.com/github/cssaivishnu/IR_Repo/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the essential libraries and mount the drive

In [ ]:
import numpy as np
import os
import shutil
import cv2
from google.colab.patches import cv2_imshow
import torchvision.transforms as transforms
from PIL import Image
import json
import random

random.seed = 20
np.random.seed = 20

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make the main repo as the current active repository

In [ ]:
os.chdir('/content/drive/MyDrive/IR_repo')
cwd = os.getcwd()
print(cwd)

From the complete dataset of approximately 63285 images from 35 categories, we consider only the product categories with atleast 150 images.

In [ ]:
dir_path = 'atlas_dataset_full'
dir_count = 0
active_dir_count = 0
complete_dir_count = 0
total_images = 0
images_list = []

for name in os.listdir(dir_path):
    if os.path.isdir(os.path.join(dir_path, name)):
        path = os.path.join(dir_path, name)
        path = os.path.join(path, 'images')
        num_files = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
        if(num_files > 150):
            active_dir_count = active_dir_count + 1
            new_path = os.path.join('dataset', name)
            os.mkdir(new_path)
            i = 0
            for f in os.listdir(path):
                if i == 600:
                    complete_dir_count = complete_dir_count + 1
                    break
                if os.path.isfile(os.path.join(path, f)):
                    i = i + 1
                    src = os.path.join(path, f)
                    fname = '{:03d}'.format(i) + '.' + f.split('.')[-1]
                    dst = os.path.join(new_path, fname)
                    shutil.copy2(src, dst)
            images_list.append(i)
        total_images = total_images + i
        dir_count += 1

print("Total Number of categories:", dir_count)
print("Number of categories with atleast 150 images:", active_dir_count)
print("Number of categories with 600 images:", complete_dir_count)
print("Total Number of Images:", total_images)

Total Number of categories: 35
Number of categories with atleast 150 images: 22
Number of categories with 600 images: 18
Total Number of Images: 19196


We find that 22 out of these 35 categories only have atleast 150 images

We want to have 600 images from each of the 22 product categories, out of which 4 of them have less than 600 images.

Now, we will perform image augmentation to increase the number of images in those 4 product categories with less than 600 images to 600 images

In [ ]:
# Here, the image is flipped horizontally to create a new image

def horizontalflipping_augmentation(path, new_path):
    # Define the horizontal flipping transformation
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=1),
    ])

    # Load the image
    img = Image.open(path)

    # Apply the horizontal flipping transformation
    img_flipped = transform(img)

    # Display the original and flipped images
    # img.show()
    # img_flipped.show()

    # Save the horizontally flipped image
    img.save(new_path)

In [ ]:
# Here, the image is modified by varying color glittering entities like contrast, brightness etc.

def colorgittering_augmentation(path,new_path):
    # Load the image
    img = cv2.imread(path)

    # Define the range of color jittering values
    brightness = 0.1
    contrast = 0.1
    saturation = 0.1
    hue = 0.1

    # Convert the image from BGR to HSV color space
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Apply color jittering to the image
    img_hsv[:, :, 2] = np.clip(img_hsv[:, :, 2] * (1 + brightness), 0, 255)
    img_hsv[:, :, 1] = np.clip(img_hsv[:, :, 1] * (1 + contrast), 0, 255)
    img_hsv[:, :, 1] = np.clip(img_hsv[:, :, 1] * (1 + saturation), 0, 255)
    img_hsv[:, :, 0] = np.clip(img_hsv[:, :, 0] * (1 + hue), 0, 255)

    # Convert the image back to BGR color space
    img_jittered = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

    # Display the original and jittered images
    # cv2_imshow(img)
    # cv2_imshow(img_jittered)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # Save the color gritted image
    cv2.imwrite(new_path, img)

In [ ]:
# Here, the image is randomly scaled to a feasible size

def randomscaling_augmentation(path,new_path):
    # Load the image
    img = Image.open(path)
    width, height = img.size
    size = int(min(width,height)*0.9)

    # Define the random scaling transformation
    transform = transforms.Compose([
        transforms.RandomResizedCrop(size=size, scale=(0.8, 1.0)),
    ])

    # Apply the random scaling transformation
    img_scaled = transform(img)

    # Display the original and scaled images
    # img.show()
    # img_scaled.show()

    # Save the randomly scaled image
    img.save(new_path)

In [ ]:
dir_path = 'dataset'
images_list = []

for name in os.listdir(dir_path):
    if os.path.isdir(os.path.join(dir_path, name)):
        path = os.path.join(dir_path, name)
        num_files = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
        images_list.append(num_files)
        if num_files != 600:
            # print(num_files)
            # print(name)
            for f in os.listdir(path):
                if os.path.isfile(os.path.join(path, f)):
                    img_name = int(f.split('.')[0])
                    for i in range(1,4):
                        new_img_name = img_name + num_files*i
                        if new_img_name > 600:
                            break
                        new_img_name = '{:03d}'.format(new_img_name) + '.' + f.split('.')[-1]
                        img_path = os.path.join(path,f)
                        new_img_path = os.path.join(path,new_img_name)
                        if i == 1:
                            horizontalflipping_augmentation(img_path,new_img_path)
                        if i == 2:
                            colorgittering_augmentation(img_path,new_img_path)
                        if i == 3:
                            randomscaling_augmentation(img_path,new_img_path)

Create the label encodings mapping the product categories and save them as JSON files

In [ ]:
dir_path = 'dataset'
i = 0
name_to_label = dict()
label_to_name = dict()

for name in os.listdir(dir_path):
    if os.path.isdir(os.path.join(dir_path, name)):
        name_to_label[name] = i
        label_to_name[i] = name
        i = i + 1

with open('name_to_label.json', 'w') as f:
    json.dump(name_to_label, f)
with open('label_to_name.json', 'w') as f:
    json.dump(label_to_name, f)

Load 500 out of 600 images from each product category and modify them to (56, 56) shape for training and evaluation of the model

In [ ]:
dir_path = 'dataset'
dataset = []
names_list = os.listdir(dir_path)
image_shape = (56, 56)

for name in names_list:
    if os.path.isdir(os.path.join(dir_path, name)):
        path = os.path.join(dir_path, name)
        # print(name_to_label[name])
        for f in os.listdir(path):
            if os.path.isfile(os.path.join(path, f)):
                num = int(f.split('.')[0])
                if num > 500:
                    continue
                img_path = os.path.join(path, f)
                img = Image.open(img_path)
                img = img.resize(image_shape, Image.ANTIALIAS)
                pixels = img.load()
                lst = []
                for i in range(img.size[0]):
                    lst1 = []
                    for j in range(img.size[1]):
                        lst1.append(list(pixels[i, j]))
                    lst.append(lst1)
                lst = np.array(lst)
                tupl = (lst,name_to_label[name])
                dataset.append(tupl)

Shuffle the dataset for randomness and split the dataset into features(X) and labels(y)

In [ ]:
random.shuffle(dataset)
X = []
y = []

for tupl in dataset:
    x, yy = tupl
    X.append(x)
    y.append(yy)
X = np.array(X)
y = np.array(y).reshape(11000,1)

# Just clear the space
dataset = []

Save the features(X) and labels(y) into numpy files so that we do not need to load the images again and again

In [ ]:
np.save('features.npy', X)
np.save('labels.npy', y)